# Environment Variable Configuration

To protect the security of API keys, we need to set them as environment variables. Environment variables allow us to access them without writing sensitive information into the code.

You can set environment variables in one of two ways:
1. Set environment variables in the terminal.
2. Set environment variables in the Python script.

Here we will set them directly in Python.


In [1]:
import os

# Set DASHSCOPE_API_KEY to your Qwen API key
os.environ['OPENAI_API_KEY'] = 'Your-Qwen-API-Key'

# 获取环境变量

使用 `os.getenv()` 函数来获取环境变量的值，这样我们可以在代码中安全地访问API密钥。

In [2]:
# Get API key
api_key = os.getenv('OPENAI_API_KEY')

# Print the key to confirm it has been successfully read
print(api_key)

Your-Qwen-API-Key


# Install Required Libraries

Next, we need to install the openai library to interact with Alibaba Cloud's large model API.

In [3]:
!pip3 install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 8.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 18.6 MB/s eta 0:00:00


# Single-Round Dialogue Demonstration

In this section, we will build a simple single-round dialogue through an API call. You can input a question, and the model will return a response.

In [4]:
from openai import OpenAI
import os

# Define a function to get model responses
def get_response():
    client = OpenAI(
        api_key=os.getenv('OPENAI_API_KEY'),  # Retrieve the API key from environment variables
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # Use Alibaba Cloud's large model API
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': '你是谁？'}
        ]
    )
    print(completion.model_dump_json())

# Call the function to engage in conversation
get_response()

{"id":"chatcmpl-d0ec996b-be56-949a-bca1-453d3a8e413f","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"我是来自阿里云的大规模语言模型，我叫通义千问。","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1734770743,"model":"qwen-turbo","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":16,"prompt_tokens":22,"total_tokens":38,"completion_tokens_details":null,"prompt_tokens_details":null}}


# Multi-Round Dialogue Demonstration

Extend the above code to support multi-round dialogues. This means the model can remember the context, thereby generating more coherent responses.

In [6]:
def get_response(messages):
    client = OpenAI(
        api_key=os.getenv('OPENAI_API_KEY'), 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=messages
    )
    return completion

# Initialize the conversation history
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# Engage in a multi-round dialogue
for i in range(3):
    user_input = input("请输入你的问题：")
    
    # Add the user's question to the conversation history
    messages.append({'role': 'user', 'content': user_input})
    
    # Get the model's response and print it
    assistant_output = get_response(messages).choices[0].message.content
    print(f'模型回复：{assistant_output}')
    
    # Add the model's response to the conversation history as well
    messages.append({'role': 'assistant', 'content': assistant_output})

请输入你的问题： 你是谁？


模型回复：我是来自阿里云的大规模语言模型，我叫通义千问。


请输入你的问题： 你可以做什么？


模型回复：我是一个能够回答问题、创作文字，还能表达观点、撰写代码的超大规模语言模型。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。


请输入你的问题： 怎么对通义千问进行fine-tune？


模型回复：对通义千问（Qwen）进行微调（Fine-tuning）通常需要访问特定的数据集和计算资源，并且需要遵循相关的使用政策和指南。作为普通用户，您可能无法直接对通义千问模型进行微调。不过，我可以告诉您一般微调过程中的几个关键步骤：

1. **准备数据集**：首先，您需要准备一个高质量的数据集来微调模型。这个数据集应该包含与您希望模型学习的任务相关的输入输出对。

2. **环境设置**：确保您的开发环境中安装了必要的库，例如TensorFlow或PyTorch等深度学习框架，以及阿里云提供的相关工具。

3. **加载预训练模型**：使用阿里云提供的API或SDK加载预训练的通义千问模型作为基础模型。

4. **数据处理**：将您的数据集转换为模型可以理解的格式，这通常涉及到分词、编码等预处理步骤。

5. **微调模型**：使用准备好的数据集来微调模型。这一步骤会根据您的需求调整模型参数，以更好地适应特定任务。

6. **评估和优化**：完成微调后，评估模型在验证集上的表现，并根据结果进一步调整模型参数或数据集，直到达到满意的性能。

7. **部署模型**：最后，您可以将微调后的模型部署到生产环境中，以便进行实际应用。

请注意，上述步骤是通用流程，具体操作可能会因阿里云的具体实现而有所不同。如果您有权限并且希望深入了解如何进行微调，建议查阅阿里云官方文档或联系技术支持获取更详细的指导。对于没有相关经验的用户来说，建议从简单的项目开始尝试，逐步积累经验。


# Stream Output Demonstration

Stream output allows us to view the model-generated responses in real-time, rather than waiting for the final result.

In [7]:
# Implement streaming output
def get_response_stream():
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                  {'role': 'user', 'content': '你是谁？'}],
        stream=True,
        stream_options={"include_usage": True}
    )
    
    # Output the generated results in real-time
    for chunk in completion:
        print(chunk.model_dump_json())

# Call the streaming output function
get_response_stream()

{"id":"chatcmpl-f863ba65-284c-9b1f-bc0e-856f13722193","choices":[{"delta":{"content":"","function_call":null,"refusal":null,"role":"assistant","tool_calls":null},"finish_reason":null,"index":0,"logprobs":null}],"created":1734770942,"model":"qwen-turbo","object":"chat.completion.chunk","service_tier":null,"system_fingerprint":null,"usage":null}
{"id":"chatcmpl-f863ba65-284c-9b1f-bc0e-856f13722193","choices":[{"delta":{"content":"我是","function_call":null,"refusal":null,"role":null,"tool_calls":null},"finish_reason":null,"index":0,"logprobs":null}],"created":1734770942,"model":"qwen-turbo","object":"chat.completion.chunk","service_tier":null,"system_fingerprint":null,"usage":null}
{"id":"chatcmpl-f863ba65-284c-9b1f-bc0e-856f13722193","choices":[{"delta":{"content":"阿里","function_call":null,"refusal":null,"role":null,"tool_calls":null},"finish_reason":null,"index":0,"logprobs":null}],"created":1734770942,"model":"qwen-turbo","object":"chat.completion.chunk","service_tier":null,"system_fing